In [1]:
import json
import os
import autogen
from autogen import ConversableAgent

In [2]:
llm_config = {"model": "llama3.1","api_type":"ollama"}

In [3]:
onboarding_personal_information_agent = ConversableAgent(
    name="onboarding_personal_information_agent",  # Removed spaces
    system_message='''You are a helpful customer onboarding agent.
    You are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

onboarding_topic_preference_agent = ConversableAgent(
    name="onboarding_topic_preference_agent",  # Removed spaces
    system_message='''You are a helpful customer onboarding agent.
    You are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config={
        "timeout":600,
        "cache_seed":42,
        "temperature":0.5,
        "config_list":llm_config,
    },
    code_execution_config=False,
    human_input_mode="NEVER",
)

customer_engagement_agent = ConversableAgent(
    name="customer_engagement_agent",  # Removed spaces
    system_message='''You are a helpful customer service agent.
    You are here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config={
        "timeout":600,
        "cache_seed":42,
        "temperature":0.5,
        "config_list":llm_config,
    },
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)


In [4]:
#Creating tasks
#Now, you can craft a series of tasks to facilitate the onboarding process.

In [5]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": "Hello, I'm here to help you get started with our product. Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return the customer information as JSON object only: {'name': '', 'location': ''}",
        },
        "max_turns": 1,
        "clear_history": True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": "Great! Could you tell me what topics you are interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history": False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

In [6]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
onboarding_personal_information_agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product. Could you tell me your name and location?

--------------------------------------------------------------------------------


C:\Anaconda\envs\AgenticAI\lib\site-packages\autogen\agentchat\chat.py:57: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Replying as customer_proxy_agent. Provide feedback to onboarding_personal_information_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Hari, Bangalore


customer_proxy_agent (to onboarding_personal_information_agent):

Hari, Bangalore

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (01f06d73-f628-484b-9f72-627baf78bcea): Maximum turns (1) reached

********************************************************************************
Starting a new chat....

********************************************************************************
onboarding_topic_preference_agent (to customer_proxy_agent):

Great! Could you tell me what topics you are interested in reading about?
Context: 
{'name': 'Hari', 'location': 'Bangalore'}

--------------------------------------------------------------------------------


Replying as customer_proxy_agent. Provide feedback to onboarding_topic_preference_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Artificial Intelligence


customer_proxy_agent (to onboarding_topic_preference_agent):

Artificial Intelligence

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (ccec1d74-8c2c-4d59-b1a4-3bfef49eb5e8): Maximum turns (1) reached

********************************************************************************
Starting a new chat....

********************************************************************************
customer_proxy_agent (to customer_engagement_agent):

Let's find something fun to read.
Context: 
{'name': 'Hari', 'location': 'Bangalore'}
I am interested in reading about Artificial Intelligence.

--------------------------------------------------------------------------------
customer_engagement_agent (to customer_proxy_agent):

Hello Hari! Bangalore, the Silicon Valley of India!

Hey, let me tell you a fascinating story about AI that I just can't wait to share with you!

Did you know that the term "Artificial Intelligence" was coined by John 

In [7]:
summary=[]
for chat_result in chat_results:
    print(chat_result.summary)
    summary.append(chat_result.summary)
    print("\n")

{'content': "{'name': 'Hari', 'location': 'Bangalore'}", 'refusal': None, 'role': 'assistant', 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': None}


{'content': 'I am interested in reading about Artificial Intelligence.', 'refusal': None, 'role': 'assistant', 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': None}


{'content': 'We discussed an introduction to Artificial Intelligence, its history, and a hypothetical scenario where an AI system learns like a human child. A fun AI-related joke was also shared.', 'refusal': None, 'role': 'assistant', 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': None}




In [8]:
summary[0]["content"]

"{'name': 'Hari', 'location': 'Bangalore'}"

In [9]:
import pandas as pd
import ast

# Input string
#data_str = "{'name': 'Ramisha', 'location': 'Coimbatore'}"
data_str=summary[0]["content"]
# Convert string to dictionary
data_dict = ast.literal_eval(data_str)

# Convert dictionary to DataFrame
df = pd.DataFrame([data_dict])

df

,name,location
0,Hari,Bangalore


In [10]:
df["TopicPreference"]=summary[1]["content"]

In [11]:
df["Enagement"]=summary[2]["content"]

In [12]:
df

,name,location,TopicPreference,Enagement
0,Hari,Bangalore,I am interested in reading about Artificial In...,We discussed an introduction to Artificial Int...
